Hamilton,W.L. *Graph Representation Learning*. 2020

# 그래프 통계량과 커널 방법

**Graph Statistics and Kernel Methods**

딥러닝과 관련된 그래프(GNN 등)를 알아보기 전에 전통적인 방식의 그래프 통계량을 알아보자.

그래프 데이터를 사용하는 전통적인 분류는 딥러닝 등장 이전의 머신러닝 패러다임을 따른다. 몇 가지 통계량 또는 특징을 추출한 다음, 머신러닝 분류기(예: 로지스틱 회귀)의 입력으로 사용한다. 

이 섹션에서는 먼저 중요한 **노드 수준의 특징과 통계량**을 소개한 다음, 이러한 노드 수준 통계량이 그래프 수준 통계량으로 일반화되고 그래프 상에서의 **커널 방법**을 설계하는 방법에 대해 논의한다.

## Node-level 통계량과 특징

### Node degree

한 노드에 연결된 엣지 수

$$d_u = \sum_{v \in V}A[u,v] \quad \text{for } u \in \mathcal{V}$$

- node degree는 고려해야 할 필수적 요소이며 전통적 기계학습 모델에서 가장 유익한 정보 중 하나이다. 
- 하지만, 한 노드와 연결된 다른 노드들의 중요성은 노드의 degree에 반영되지 않는다. 즉, 두 노드가 같은 degree를 갖더라도 이웃의 중요도에 따라 중요도가 다를 수 있다.

### Node centrality

*eigenvector centrality*

- node degree와 달리, **노드의 이웃들의 중요도**까지 고려하는 방법이다.

$$e_u = \frac{1}{\lambda}\sum_{v \in V}A[u,v] e_v, \quad \forall u \in \mathcal{V}$$

- 식처럼 recursive한 성질을 가진다. 
- 단, 위의 식을 다시 쓰면 벡터 $\mathbf{e}$에 대해 $\lambda \mathbf{e} = A \mathbf{e}$와 같다. 즉, centrality는 인접행렬의 eigenvector이다. 양의 centrality 값을 가정하면, Perron-Frobenius Theorem을 적용하여 $\mathbf{e}$를 구할 수 있다.

- 또한, eigenvector centrality느 그래프에서 무한 랜덤워킹할 때 해당 노드에 도착할 확률이라고도 볼 수 있다.
    - $\mathbf{e}^{(t+1)} = A \mathbf{e}^{(t)}$ : power iteration

다른 centrality

- *betweenness centrality*: 한 노드가 다른 두 노드 사이의 최단 경로에 있을 빈도

- *closeness centrality*: 한 노드와 다른 모든 노드 사이 최단거리들의 평균

### Clustering Coefficient

한 노드의 로컬(local) 이웃에서의 closed triangle의 비율

- clustering coefficient의 변형 중 'local' variant를 보자.
$$c_u = \frac{|(v_1,v_2) \in \mathcal{E}: v_1, v_2 \in \mathcal{N}(u)|}{\binom{d_u}{2}}$$
    - 분자는 노드 $u$의 이웃 간 엣지 수이고, 분모는 $u$의 이웃에 있는 노드 쌍의 수이다.

- 노드의 이웃이 얼마나 tight하게 클러스터되어 있는지를 측정한다.
- clustering coefficient가 1이면, $u$의 모든 이웃은 서로의 이웃이기도 하게 된다.

### Closed Triangles, Ego Graphs, and Motif

- clustering coefficient는 노드의 로컬 이웃 사이에 있는 **closed triangle**의 수라고 볼 수도 있다.

- clustering coefficient는 실제 triangle의 수와 노드의 '**ego graph**'의 총 가능 triangle 수의 비이기도 하다. 
     - ego graph: 노드, 이웃, 이웃 간 모든 엣지를 포함하는 subgraph
     
- 노드의 ego graph 내 임의의 motif나 graphlet을 세는 개념으로 일반화될 수 있다. 즉, 단순히 triangle을 세는 것보다는 특정 길이의 순환과 같은 복잡한 구조를 고려할 수 있고, 이러한 서로 다른 motif가 ego graph에서 얼마나 자주 발생하는지 세는 것으로 노드를 특성화할 수 있다. 

- 이러한 방식으로 노드의 ego graph를 조사하여 노드 레벨의 통계량을 계산하는 작업을 그래프 레벨로 근본적으로 변환할 수 있다.

## Graph-level 특징과 Graph Kernels

### Bag of Nodes
- 단순히 node-level의 통계량을 집계하여 graph-level 피쳐로 만든다. 
- 하지만, 로컬(local) node-level의 정보만에 기반하기 때문에 글로벌(global)한 속성을 놓칠 수 있다.

### The Weisfieler-Lehman Kernel

- 노드-레벨 피처를 더 풍성하게 만들고나서 집계하는 방법 중 하나이다.

- WL 알고리즘은 다음과 같다.
    - 1. 각 노드에 초기 레이블 값을 준다. 보통 degree로 한다.: $l^{(0)}(v) = d_v$
    - 2. 노드의 이웃에서 현재 레이블의 multi-set을 hashing하여 새로운 레이블을 할당하며, 이를 반복한다.: $$l^{(i)}(v) = \text{HASH}(\{ \{ l^{(i)}(v) \quad \forall u \in \mathcal{N}(v) \} \} )$$
        - {{ }}는 mult-set을 뜻한다. multi-set은 원소의 중복을 허용한다.
    - 3. 이를 $K$번 반복하면 각 노드마다 $l^{(K)}(v)$가 생기며, 이는 $K$-hop 이웃의 구조를 요약한다고 볼 수 있다. 다시 말해, WL kernel은 두 그래프의 결과 레이블 집합 간의 차이를 측정하는 것이다.
    
- 이러한 WL kernel은 그래프 동형(isomorphism)을 확인하는 데 쓰인다: 두 그래프가 $K$번 WL알고리즘 반복 뒤 생기는 레이블 집합이 같은지를 확인한다.

### Graphlets, Path-Based Methods

**graphlets**

<img src="img/fig2_2.png" width='500'>

> Figure 2.2 in Hamilton,*Graph Representation Learning*. 2020: 크기가 3인 graphlet들

- 그래프에 있는 graphlets (서로 다른 작은 subgraph 구조) 수
- graphlet kernel은 전체 그래프에서 특정한 크기의 그래프 구조가 얼마나 나타나는지 세는 방식이다. 하지만 모든 graphlet을 세는 것은 번거로운 일이다.

*path-based 방법*

- graphlet을 세는 방법 대신 그래프에서 서로 다른 경로(path)의 종류가 얼마나 있는지를 본다.
- 예를 들어 그래프에서 랜덤워크를 가정할때 발생하는 서로 다른 degree 시퀀스를 카운팅하는 random walk kernel(Kashima et al.(2003)), 또는 가장짧은 경로만을 카운팅하는 shortest-path kernel(Borgwardt and Kriegel (2005)) 등이 있다.

# 참고자료

[1] Hamilton, W. L. (2020). *Graph Representation Learning.* Morgan & Claypool Publishers.